# SQL Проект

Цель исследования: с помощью SQL-запросов ответить на ряд вопросов, характеризующих базу данных книжной сферы.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследование таблиц - вывести первые строки

In [2]:
query = '''
           SELECT *
           FROM books
           LIMIT 1
        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

   book_id  author_id         title  num_pages publication_date  publisher_id
0        1        546  'Salem's Lot        594       2005-11-01            93


In [3]:
query = '''
           SELECT *
           FROM authors
           LIMIT 1
        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

   author_id      author
0          1  A.S. Byatt


In [4]:
query = '''
           SELECT *
           FROM ratings
           LIMIT 1
        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

   rating_id  book_id    username  rating
0          1        1  ryanfranco       4


In [5]:
query = '''
           SELECT *
           FROM reviews
           LIMIT 1
        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

   review_id  book_id      username  \
0          1        1  brandtandrea   

                                                text  
0  Mention society tell send professor analysis. ...  


In [6]:
query = '''
           SELECT *
           FROM publishers
           LIMIT 1
        '''
df = pd.io.sql.read_sql(query, con = engine)
print(df)

   publisher_id publisher
0             1       Ace


Наша база данных состоит из 5 таблиц, соединённых между собой с помощью ключей. Таблица books - содержит данные о книгах; authors - об авторах; publishers - об издательствах; ratings - о пользовательских оценках книг; reviews - пользовательских обзорах на книги.

## Посчитайте, сколько книг вышло после 1 января 2000 года;

In [7]:
query = '''
           select count(title) as Количество
           from books
           where publication_date > '2000-01-01'
        '''

In [8]:
df = pd.io.sql.read_sql(query, con = engine)

In [9]:
print(df)

   Количество
0         819


В нашей базе данных есть информация о 819 книгах после 1 января 2000

## Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [10]:
query = '''
           select books.title as Название, round(ratings.avg_rating, 2) as Рейтинг, reviews.review_cnt as Рецензии
           from books 
           
           left join 
           (select book_id, avg(rating) as avg_rating 
           from ratings 
           group by book_id) as ratings on books.book_id = ratings.book_id
           
           left join
           (select book_id, count(text) as review_cnt
           from reviews 
           group by book_id) as reviews on books.book_id = reviews.book_id
        '''

In [11]:
df = pd.io.sql.read_sql(query, con = engine)

In [12]:
print(df)

                                              Название  Рейтинг  Рецензии
0            The Body in the Library (Miss Marple  #3)     4.50       2.0
1                                            Galápagos     4.50       2.0
2                             A Tree Grows in Brooklyn     4.25       5.0
3    Undaunted Courage: The Pioneering First Missio...     4.00       2.0
4                                          The Prophet     4.29       4.0
..                                                 ...      ...       ...
995                                Alice in Wonderland     4.23       4.0
996          A Woman of Substance (Emma Harte Saga #1)     5.00       2.0
997                                          Christine     3.43       3.0
998  The Magicians' Guild (Black Magician Trilogy  #1)     3.50       2.0
999                           The Plot Against America     3.00       2.0

[1000 rows x 3 columns]


В итоговой таблице у нас есть информация по каждой книге в разрезе её средней оценки и количества обзоров. Можно увидеть, даже без подробного анализа, что скорее всего средняя оценка выставляется по 5-балльной шкале.

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [13]:
query = '''
           select publisher as Издательство, count(title)
           
           from publishers as p join books as b on p.publisher_id = b.publisher_id
           
           WHERE b.num_pages > 50
           group by publisher
           order by count desc
           limit 1
        '''

In [14]:
df = pd.io.sql.read_sql(query, con = engine)

In [15]:
print(df)

    Издательство  count
0  Penguin Books     42


Penguin Books - издательство, выпустившее больше всего книг, толще 50 страниц. 42 книги, кажется, что немного, но может это только "кажется".

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [16]:
query = '''select author as Автор, avg(r.rating)

        from authors as a join books as b on a.author_id = b.author_id 
        join ratings as r on b.book_id = r.book_id
        
        Where r.book_id in (select book_id
        from ratings
        group by book_id
        having count(rating) >= 50)
        
        group by author
        order by avg desc
        limit 1
        '''

In [17]:
df = pd.io.sql.read_sql(query, con = engine)

In [18]:
print(df)

                        Автор       avg
0  J.K. Rowling/Mary GrandPré  4.287097


Самая высокая средняя оценка у Дж. Роулинг - автора книг о Гарри Поттере (4.29 средняя оценка)

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [19]:
query = ''' select count(text)/count(DISTINCT username) as avg_rating
            from reviews
            Where username in (select username
            from ratings
            group by username
            having count(username) > 50)
        '''

In [20]:
df = pd.io.sql.read_sql(query, con = engine)

In [21]:
print(df)

   avg_rating
0          24


Среднее количество обзоров у пользователей, поставивишх более 50 оценок - 24. Видимо, делать рецензии на книги - это у них хобби :)

## Итоговый вывод:

С помощью SQL-запросов мы выяснили, что в нашей базе данных есть информация о 819 книгах выпущенных после 1 января 2000.<br> Для каждой книги есть информация о средней оценке по 5-балльной шкале и количестве рецензий (обзоров). Так самый высокий рейтинг из популярных писателей (у кого больше 50 книг) у Дж. Роулинг - автора книг о Гарри Поттере (4.29 средняя оценка). Самое "распространённое" издательство в нашей базе - Penguin Books, оно выпустило 42 книги объёмом больше 50 страниц.